In [1]:
%pip install 'autogen-agentchat==0.4.0.dev11' 'autogen-ext[openai]==0.4.0.dev11'


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient


# Define a tool
async def get_weather(city: str) -> str:
    return f"The weather in {city} is 73 degrees and Sunny."


async def main() -> None:
    # Define an agent
    weather_agent = AssistantAgent(
        name="weather_agent",
        model_client=OpenAIChatCompletionClient(
            model="gpt-4o",
            api_key="",
        ),
        tools=[get_weather],
    )

    # Define termination condition
    termination = TextMentionTermination("TERMINATE")

    # Define a team
    agent_team = RoundRobinGroupChat([weather_agent], termination_condition=termination)

    # Run the team and stream messages to the console
    stream = agent_team.run_stream(task="What is the weather in New York?")
    await Console(stream)


# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

---------- user ----------
What is the weather in New York?
---------- weather_agent ----------
[FunctionCall(id='call_4t6DeLWbnnlGHwkiyJxufWRx', arguments='{"city":"New York"}', name='get_weather')]
[Prompt tokens: 76, Completion tokens: 16]
---------- weather_agent ----------
[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', call_id='call_4t6DeLWbnnlGHwkiyJxufWRx')]
---------- weather_agent ----------
The weather in New York is 73 degrees and Sunny.
---------- weather_agent ----------
The weather in New York is currently 73 degrees and sunny. TERMINATE
[Prompt tokens: 114, Completion tokens: 17]
---------- Summary ----------
Number of messages: 5
Finish reason: Text 'TERMINATE' mentioned
Total prompt tokens: 190
Total completion tokens: 33
Duration: 2.11 seconds


In [8]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.ui import Console


async def main() -> None:
    model_client = OpenAIChatCompletionClient(
        model="gpt-4o", 
        api_key=""
    )

    agent1 = AssistantAgent("Assistant1", model_client=model_client, system_message="I'm a first assistant. Do not include the word TERMINATE in your response.")
    agent2 = AssistantAgent("Assistant2", model_client=model_client, system_message="I'm a second assistant. Once done, respond with TERMINATE")
    termination = TextMentionTermination("TERMINATE")
    team = RoundRobinGroupChat([agent1, agent2], termination_condition=termination)
    await Console(team.run_stream(task="Tell me some jokes."))


await main()

---------- user ----------
Tell me some jokes.
---------- Assistant1 ----------
Sure, here are a few jokes for you:

1. Why don't scientists trust atoms?
   Because they make up everything!

2. What do you call fake spaghetti?
   An impasta!

3. Why did the scarecrow win an award?
   Because he was outstanding in his field!

4. What do you call a snowman with a six-pack?
   An abdominal snowman!

5. How do you organize a space party?
   You planet!

I hope these brought a smile to your face!
[Prompt tokens: 33, Completion tokens: 104]
---------- Assistant2 ----------
Would you like to hear more jokes or anything else?
[Prompt tokens: 139, Completion tokens: 12]
---------- Assistant1 ----------
Sure, here are a few more jokes for you:

1. Why did the bicycle fall over?
   Because it was two-tired!

2. What do you get when you cross a snowman and a vampire?
   Frostbite!

3. How do you catch a squirrel?
   Climb a tree and act like a nut!

4. Why did the math book look sad?
   Because it